In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
import warnings
import xgboost as xgb

In [2]:
pd.options.mode.use_inf_as_na = True
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('G:\IDS2018\datasets\Thursday-15-02-2018_TrafficForML_CICFlowMeter.csv')

In [4]:
numerical_fea = list(data.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea, list(data.columns)))

In [5]:
def get_numerical_serial_fea(data, feas):
    numerical_serial_fea = []
    numerical_noserial_fea = []
    for fea in feas:
        temp = data[fea].nunique()
        if temp <= 10:
            numerical_noserial_fea.append(fea)
        else:
            numerical_serial_fea.append(fea)
    return numerical_serial_fea, numerical_noserial_fea
numerical_serial_fea, numerical_noserial_fea = get_numerical_serial_fea(data, numerical_fea)

In [6]:
data[numerical_serial_fea] = data[numerical_serial_fea].fillna(data[numerical_serial_fea].mean())

In [7]:
one_value_fea = [col for col in data.columns if data[col].nunique() <= 1]
one_value_fea

['Bwd PSH Flags',
 'Fwd URG Flags',
 'Bwd URG Flags',
 'CWE Flag Count',
 'Fwd Byts/b Avg',
 'Fwd Pkts/b Avg',
 'Fwd Blk Rate Avg',
 'Bwd Byts/b Avg',
 'Bwd Pkts/b Avg',
 'Bwd Blk Rate Avg']

In [8]:
data.drop(columns=one_value_fea, inplace=True)

In [9]:
data['Label'].replace(to_replace='Benign', value=0, inplace=True)
data['Label'].replace(to_replace='DoS attacks-GoldenEye', value=1, inplace=True)
data['Label'].replace(to_replace='DoS attacks-Slowloris', value=1, inplace=True)

In [10]:
data

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,0,0,15/02/2018 08:25:18,112641158,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.026633,5.632058e+07,7.042784e+02,56321077,56320081,112641158,5.632058e+07,7.042784e+02,56321077,56320081,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320579.0,7.042784e+02,56321077,56320081,0
1,22,6,15/02/2018 08:29:05,37366762,14,12,2168,2993,712,0,154.857143,254.855270,976,0,249.416667,395.929392,1.381174e+02,0.695806,1.494670e+06,3.894924e+06,15617415,7,37366762,2.874366e+06,5.104444e+06,15617415,40,37366730,3.396975e+06,5.564224e+06,15936762,893,0,456,416,0.374665,0.321141,0,976,191.148148,320.122898,102478.669516,0,0,0,1,0,0,0,0,198.500000,154.857143,249.416667,14,2168,12,2993,29200,232,8,32,1024353.0,649038.754495,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,0
2,47514,6,15/02/2018 08:29:42,543,2,0,64,0,64,0,32.000000,45.254834,0,0,0.000000,0.000000,1.178637e+05,3683.241252,5.430000e+02,0.000000e+00,543,543,543,5.430000e+02,0.000000e+00,543,543,0,0.000000e+00,0.000000e+00,0,0,1,64,0,3683.241252,0.000000,0,64,42.666667,36.950417,1365.333333,0,1,0,0,1,0,0,0,64.000000,32.000000,0.000000,2,64,0,0,244,-1,0,32,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,0
3,0,0,15/02/2018 08:28:07,112640703,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.026633,5.632035e+07,3.669884e+02,56320611,56320092,112640703,5.632035e+07,3.669884e+02,56320611,56320092,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320351.5,3.669884e+02,56320611,56320092,0
4,0,0,15/02/2018 08:30:56,112640874,3,0,0,0,0,0,0.000000,0.000000,0,0,0.000000,0.000000,0.000000e+00,0.026633,5.632044e+07,7.198347e+02,56320946,56319928,112640874,5.632044e+07,7.198347e+02,56320946,56319928,0,0.000000e+00,0.000000e+00,0,0,0,0,0,0.026633,0.000000,0,0,0.000000,0.000000,0.000000,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,3,0,0,0,-1,-1,0,0,0.0,0.000000,0,0,56320437.0,7.198347e+02,56320946,56319928,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,50111,6,15/02/2018 09:04:42,22,3,0,31,0,31,0,10.333333,17.897858,0,0,0.000000,0.000000,1.409091e+06,136363.636364,1.100000e+01,1.555635e+01,22,0,22,1.100000e+01,1.555635e+01,22,0,0,0.000000e+00,0.000000e+00,0,0,0,60,0,136363.636364,0.000000,0,31,7.750000,15.500000,240.250000,0,0,0,0,1,0,0,0,10.333333,10.333333,0.000000,3,31,0,0,64,-1,1,20,0.0,0.000000,0,0,0.0,0.000000e+00,0,0,0
1048571,443,6,15/02/2018 09:03:55,54682783,5,1,123,46,46,0,24.600000,23.276598,46,46,46.000000,0.000000,3.090552e+00,0.109724,1.093656e+07,2.436610e+07,54523813,17,54682783,1.367070e+07,2.723551e+07,54523813,17,0,0.000000e+00,0.000000e+00,0,0,1,100,20,0.09

In [11]:
features = [f for f in data.columns if f in ['Fwd Seg Size Min', 'Bwd IAT Mean', 'Init Fwd Win Byts', 'Bwd IAT Min', 
                                              'Flow IAT Std', 'Flow IAT Max', 'Bwd IAT Max', 'Bwd IAT Std', 'Idle Max', 
                                              'Fwd Pkt Len Std', 'Idle Std', 'Idle Min', 'Fwd Header Len', 'Pkt Len Max', 
                                              'Dst Port', 'Fwd Pkts/s', 'Flow Duration', 'Flow Byts/s', 'Flow Pkts/s', 
                                              'Fwd IAT Tot', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
                                              'Bwd IAT Tot', 'Bwd IAT Min', 'Pkt Len Var', 'Idle Mean']]

In [12]:
x_train = data[features]
y_train = data['Label']

In [13]:
def cv_model(clf, x_train, y_train, clf_name):
    '''
        @param clf: 分类器包名(lgb&xgb)
        @param x_train: 训练集X
        @param y_train: 训练集y
        @param clf_name: 分类器名：'lgb' 'xgb'

        @return: model 最后一次交叉训练后的模型
        @return: train 分类器在训练集上的预测结果
    '''

    folds = 5
    seed = 2020
    # use StratifiedKFold instead of KFold to solve unbalanced datasets
    kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(x_train.shape[0])

    cv_accuracy_scores = []
    cv_f1_scores = []
    cv_auc_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(x_train, y_train)):
        print('************************************ {0}/{1} ************************************'.format(str(i+1), folds))
        trn_x, trn_y, val_x, val_y = x_train.iloc[train_index], y_train[train_index], x_train.iloc[valid_index], y_train[valid_index]
        
        if clf_name == "lgb":
            train_matrix = clf.Dataset(trn_x, label=trn_y)
            valid_matrix = clf.Dataset(val_x, label=val_y)

            params = {
                'boosting_type': 'gbdt',
                'objective': 'binary',
                'metric': 'auc',
                'min_child_weight': 5,
                'num_leaves': 2 ** 5,
                'lambda_l2': 10,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'bagging_freq': 4,
                'learning_rate': 0.1,
                'seed': 2020,
                'nthread': 28,
                'n_jobs':24,
                'silent': True,
                'verbose': -1,
                'is_unbalance': True, # unblanced datasets
            }

            model = clf.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix], 
                              verbose_eval=200, early_stopping_rounds=200)
            val_score = model.predict(val_x, num_iteration=model.best_iteration)
            
            # 输出前20特征重要性从高到低
            print(list(sorted(zip(model.feature_name(), model.feature_importance("gain")), key=lambda x: x[1], reverse=True))[:20])
                
        if clf_name == "xgb":
            train_matrix = clf.DMatrix(trn_x , label=trn_y)
            valid_matrix = clf.DMatrix(val_x , label=val_y)
            
            params = {'booster': 'gbtree',
                      'objective': 'binary:logistic',
                      'eval_metric': 'auc',
                      'gamma': 1,
                      'min_child_weight': 1.5,
                      'max_depth': 5,
                      'lambda': 10,
                      'subsample': 0.7,
                      'colsample_bytree': 0.7,
                      'colsample_bylevel': 0.7,
                      'eta': 0.04,
                      'tree_method': 'exact',
                      'seed': 2020,
                      'nthread': 36,
                      "silent": True,
                      'is_unbalance': True, # unblanced datasets
                      }
            
            watchlist = [(train_matrix, 'train'),(valid_matrix, 'eval')]
            
            model = clf.train(params, train_matrix, num_boost_round=50000, evals=watchlist, verbose_eval=200, early_stopping_rounds=200)
            val_score  = model.predict(valid_matrix, ntree_limit=model.best_ntree_limit)

        # 交叉验证每次得到一部分结果
        val_pred = np.int8(np.round(val_score))
        train[valid_index] = val_pred
        
        cv_accuracy_scores.append(accuracy_score(val_y, val_pred))
        cv_f1_scores.append(f1_score(val_y, val_pred))
        cv_auc_scores.append(roc_auc_score(val_y, val_score))
        
        print(cv_accuracy_scores)
        print(cv_f1_scores)
        print(cv_auc_scores)
       
    print('%s_accuracy_score_list:' % clf_name, cv_accuracy_scores)
    print('%s_accuracy_score_mean:' % clf_name, np.mean(cv_accuracy_scores))
    print('%s_accuracy_score_std:' % clf_name, np.std(cv_accuracy_scores))
    
    print('%s_f1_score_list:' % clf_name, cv_f1_scores)
    print('%s_f1_score_mean:' % clf_name, np.mean(cv_f1_scores))
    print('%s_f1_score_std:' % clf_name, np.std(cv_f1_scores))
    
    print('%s_auc_score_list:' % clf_name, cv_auc_scores)
    print('%s_auc_score_mean:' % clf_name, np.mean(cv_auc_scores))
    print('%s_auc_score_std:' % clf_name, np.std(cv_auc_scores))
    
    return model, train

In [14]:
def lgb_model(x_train, y_train):
    lgb_trained_model, lgb_train = cv_model(lgb, x_train, y_train, 'lgb')
    return lgb_trained_model, lgb_train
def xgb_model(x_train, y_train):
    xgb_trained_model, xgb_train = cv_model(xgb, x_train, y_train, 'xgb')
    return xgb_trained_model, xgb_train

In [15]:
lgb_trained_model, lgb_train = lgb_model(x_train, y_train)

************************************ 1/5 ************************************
[LightGBM] [Warning] num_threads is set with nthread=28, will be overridden by n_jobs=24. Current value: num_threads=24
[LightGBM] [Warning] Unknown parameter: silent
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 1	valid_1's auc: 1
[400]	training's auc: 1	valid_1's auc: 1
[600]	training's auc: 1	valid_1's auc: 1
Early stopping, best iteration is:
[588]	training's auc: 1	valid_1's auc: 1
[('Fwd_Seg_Size_Min', 8435543.303896904), ('Dst_Port', 429830.008274436), ('Init_Fwd_Win_Byts', 43254.188398227096), ('Fwd_Pkts/s', 17958.392886511167), ('Fwd_Header_Len', 13427.375798285007), ('Flow_Duration', 5278.884028681088), ('Flow_IAT_Max', 5155.4699004832655), ('Pkt_Len_Max', 4032.890529476106), ('Fwd_IAT_Min', 3442.290228003869), ('Flow_Pkts/s', 1987.5318608304951), ('Fwd_IAT_Tot', 1547.2342333677225), ('Fwd_IAT_Max', 1364.1167092528194), ('Fwd_IAT_Mean', 608.9635840267874), ('Flo

In [16]:
xgb_trained_model, xgb_train = xgb_model(x_train, y_train)

************************************ 1/5 ************************************
[20:05:32] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { is_unbalance, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.99458	eval-auc:0.99468
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 200 rounds.
[200]	train-auc:1.00000	eval-auc:1.00000
[400]	train-auc:1.00000	eval-auc:1.00000
Stopping. Best iteration:
[241]	train-auc:1.00000	eval-auc:1.00000

[0.9999475478625754]
[0.9994760158147955]
[0.9999996246836317]
************************************ 2/5 ************************************
[20:09:26] WARNING: C:\Users\Administrator\workspace